In [1]:
import skrub
from sklearn.ensemble import HistGradientBoostingClassifier
from skrub import TableVectorizer

import gyyre

dataset = skrub.datasets.fetch_midwest_survey()

responses = skrub.var("response", dataset.X)
responses = responses.skb.subsample(n=100).skb.set_description(dataset.metadata["description"])

labels = skrub.var("labels", dataset.y)
labels = labels.skb.subsample(n=100).skb.set_name(dataset.metadata["target"])

responses = responses.skb.mark_as_X()
labels = labels.skb.mark_as_y()

responses_with_additional_features = responses.with_sem_features(
    nl_prompt="""
        Compute additional demographics-related features, use your intrinsic knowledge about the US. 
        Take into account how the identification with the country or regions of it changed over the generations.         
        Also think about how the identification differs per class and education. The midwest is generally associated 
        with "Midwestern values" — friendliness, modesty, hard work, and community-mindedness.
    """,
    name="demographic_features",
    how_many=5,
)

feature_encoder = TableVectorizer()
encoded_responses = responses_with_additional_features.skb.apply(feature_encoder)

learner = HistGradientBoostingClassifier()
model = encoded_responses.skb.apply(learner, y=labels)

--- Fitting gyyre.with_sem_features('Compute additional demographics...', 5)
	> Querying 'openai/gpt-4.1' with 2 messages...'
	> Computed 5 new feature columns: ['Age_Group_Generation', 'Education_Level_Num', 'Income_Bracket_Num', 'Is_Midwest_Resident', 'Midwest_Identification_Score'], removed 0 feature columns: []


In [2]:
memory, states = gyyre.greedy_optimise_semantic_operator(
    model,
    "demographic_features",
    num_iterations=5,
    scoring="accuracy",
    cv=3,
)

--- COMPUTING DAG SUMMARY for context-aware optimisation ---
---ITERATION 0 -> Fitting with memory ---
--- Fitting gyyre.with_sem_features('Compute additional demographics...', 5) for a classification task, predicting `Census_Region` with sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier
	> Querying 'openai/gpt-4.1' with 2 messages...'
	> Computed 5 new feature columns: ['Age_Group_Generation', 'Education_Level_Num', 'Gender_Binary', 'Household_Income_Num', 'Midwest_Identity_Score'], removed 0 feature columns: []
---ITERATION 0 -> Evaluation ---
--- Using provided state for gyyre.with_sem_features('Compute additional demographics...', 5)
--- Using provided state for gyyre.with_sem_features('Compute additional demographics...', 5)
--- Using provided state for gyyre.with_sem_features('Compute additional demographics...', 5)
---ITERATION 0 -> accuracy: 0.7265878112561325
---ITERATION 1 -> Fitting with memory ---
--- Fitting gyyre.with_sem_features('

In [3]:
highest_accuracy = 0.0
corresponding_state = None

for memory_line, state in zip(memory, states):
    if memory_line["score"] > highest_accuracy:
        highest_accuracy = memory_line["score"]
        corresponding_state = state

print(f"Highest accuracy: {highest_accuracy}\n\n")
print("\n".join(corresponding_state["generated_code"]))

Highest accuracy: 0.749031422598661


# (Feature: Age_Group_Generation - Map age ranges to generational cohorts)
# Usefulness: Generational identity influences regional self-identification and values, which can help distinguish Census Regions. For example, Baby Boomers may have different regional attachments than Millennials.
# Input samples: 'Age': ['18-29', '30-44', '45-60']
def map_age_to_generation(age):
    if pd.isnull(age):
        return np.nan
    if age in ['18-29']:
        return 'Gen Z/Millennial'
    elif age in ['30-44']:
        return 'Millennial/Gen X'
    elif age in ['45-60']:
        return 'Gen X/Boomer'
    elif age in ['60+']:
        return 'Boomer+'
    else:
        return np.nan
df['Age_Group_Generation'] = df['Age'].apply(map_age_to_generation)

# (Feature: Education_Level_Num - Ordinal encoding of education level)
# Usefulness: Education level is correlated with mobility, regional identity, and values, which can help the classifier distinguish between Cens